In [22]:
import numpy as np 
import tensorflow as tf
import pickle
from tensorflow.keras import  preprocessing, utils
from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers.recurrent import LSTM,SimpleRNN
import yaml
import os
import pandas as pd
# from numba import jit, cuda
import pickle
from statistics import mode

In [23]:
tokenizer = preprocessing.text.Tokenizer()
def token(questions , answers):
    # tokenizer = preprocessing.text.Tokenizer()
    tokenizer.fit_on_texts( questions + answers )
    VOCAB_SIZE = len( tokenizer.word_index )+1
    # print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))


    vocab = []
    for word in tokenizer.word_index:
        vocab.append(word)
    
    maxlen_questions = int(np.ceil(np.mean([len(q) for q in questions])))
    maxlen_answers = int(np.ceil(np.mean([len(a) for a in answers])))
    
    print(maxlen_questions , maxlen_answers)
    tokenized_questions = tokenizer.texts_to_sequences( questions  )
    padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions, maxlen = maxlen_questions, padding = 'post' ,truncating= 'post')
    encoder_input_data = np.array(padded_questions)


    tokenized_answers = tokenizer.texts_to_sequences( answers  )
    padded_answers = np.array(preprocessing.sequence.pad_sequences( tokenized_answers , maxlen= maxlen_answers , padding='post',truncating= 'post' ))
    decoder_input_data = np.array( padded_answers )


    for i in range(len(tokenized_answers)) :
        tokenized_answers[i] = tokenized_answers[i][1:]
    padded_answers1 = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post',truncating= 'post'  )
    # onehot_answers = utils.to_categorical( padded_answers1 , num_classes= VOCAB_SIZE ,dtype = 'float32' )
    decoder_output_data = np.array( padded_answers1 )
    return VOCAB_SIZE ,maxlen_questions, maxlen_answers,encoder_input_data ,decoder_input_data ,decoder_output_data

In [24]:
with open ('../processed_dataset/sad.pickle','rb+') as f: 
  questions , answers =  pickle.load(f)

In [25]:
VOCAB_SIZE, maxlen_questions, maxlen_answers, encoder_input_data, decoder_input_data, decoder_output_data = token(questions , answers)

68 80


In [26]:
print(VOCAB_SIZE, maxlen_questions, maxlen_answers, encoder_input_data.shape, decoder_input_data.shape, decoder_output_data.shape)

6281 68 80 (8144, 68) (8144, 80) (8144, 80)


In [27]:
def custom_loss_function(y_true, y_pred):
   y1 = tf.numpy_function(utils.to_categorical , [y_true ,VOCAB_SIZE] , tf.float32) 
   cce = tf.keras.losses.CategoricalCrossentropy()
   # y2 = utils.to_categorical(y_true , VOCAB_SIZE )
   # print(y1[0] , y2[0])
   loss = cce(y1, y_pred)
   # del y1
   return loss

In [28]:
# encoder_inputs = tf.keras.layers.Input(shape=( maxlen_questions , ))
# encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE ,200, mask_zero=True ) ( encoder_inputs )
# encoder_lstm1 = tf.keras.layers.LSTM ( 200 , return_state=True , return_sequences=True)
# encoder_lstm2 = tf.keras.layers.LSTM ( 200 , return_state=True , return_sequences=True)
# encoder_output1 , state_h1 , state_c1 = encoder_lstm1(encoder_embedding )
# encoded_state1 = [state_h1 , state_c1]
# encoder_output2 , state_h2 , state_c2 = encoder_lstm2(encoder_output1)
# encoded_state2 = [state_h2 , state_c2]

# decoder_inputs = tf.keras.layers.Input(shape=( maxlen_answers ,  ))
# decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE ,200, mask_zero=True ) ( decoder_inputs )
# decoder_lstm1 = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True)
# decoder_lstm2 = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True)
# decoder_output1 ,state_h3 , state_c3 = decoder_lstm1 (decoder_embedding ,  initial_state=encoded_state1)
# decoder_state21 = [state_h3 , state_c3]
# decoder_lstm_output1 = [decoder_output1]
# decoder_output2 ,state_h4 ,state_c4  = decoder_lstm2 (decoder_output1 ,  initial_state=encoded_state2)
# decoder_state2 = [state_h4 , state_c4]
# decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax)
# output = decoder_dense (decoder_output2)

encoder_inputs = tf.keras.layers.Input(shape=( maxlen_questions , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( maxlen_answers ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )
model3 = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model3.compile(optimizer=tf.keras.optimizers.RMSprop(), loss=custom_loss_function , run_eagerly= False )
model3.summary()

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 68)]         0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           [(None, 80)]         0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 68, 200)      1256200     input_13[0][0]                   
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 80, 200)      1256200     input_14[0][0]                   
___________________________________________________________________________________________

In [29]:
model3.load_weights('sad.h5')

In [8]:
model3.fit([encoder_input_data, decoder_input_data], decoder_output_data, batch_size= 50, epochs= 50 ) 
model3.save( 'sad.h5' )

Epoch 1/50
163/163 [==============================] - 107s 423ms/step - loss: 1.5254
Epoch 2/50
163/163 [==============================] - 68s 422ms/step - loss: 1.4556
Epoch 3/50
163/163 [==============================] - 76s 461ms/step - loss: 1.3980
Epoch 4/50
163/163 [==============================] - 63s 386ms/step - loss: 1.3419
Epoch 5/50
163/163 [==============================] - 68s 411ms/step - loss: 1.2876
Epoch 6/50
163/163 [==============================] - 69s 423ms/step - loss: 1.2345
Epoch 7/50
163/163 [==============================] - 68s 417ms/step - loss: 1.1827
Epoch 8/50
163/163 [==============================] - 52s 317ms/step - loss: 1.1312
Epoch 9/50
163/163 [==============================] - 20s 124ms/step - loss: 1.0802
Epoch 10/50
163/163 [==============================] - 20s 125ms/step - loss: 1.0294
Epoch 11/50
163/163 [==============================] - 20s 125ms/step - loss: 0.9787
Epoch 12/50
163/163 [==============================] - 20s 125ms/step - l

In [31]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    
    decoder_states = [state_h, state_c]

    decoder_outputs = decoder_dense(decoder_outputs)
    
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model
def str_to_tokens( sentence : str ):

    words = sentence.lower().split()
    tokens_list = list()
  
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')
enc_model , dec_model = make_inference_models()

for _ in range(10):
    ads = str_to_tokens( input( 'Enter question : ' ))
    print(ads)
    states_values = enc_model.predict( ads ) 
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        print(sampled_word_index)
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )

[[882   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
8
13
14
61
2
 that is so sad end
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
1
15
60
19
10
144
15
60
19
45
423
132
11
178
17
67
1
169
1
123
15
4
23
10
71
11
1
15
4
23
10
482
11
26
52
37
62
23
75
43
145
2
 i have been in my must have been in your past year and kind of one i guess i could have to be my yeah and i have to be my god and just all really don't be hard at myself end
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
1
15
60
19
10
144
15
60
19
45
423
132
11
178
17
67
1
169
1
123
15
4
23
10
71
11
1
15
4
23
10
482
11